In [5]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [6]:
!pip install xlrd==1.2.0

In [7]:
# Loading the Data
data = pd.read_excel('Online_Retail.xls')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [8]:

# Exploring the columns of the data
data.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [9]:

# Exploring the different regions of transactions
data.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland'], dtype=object)

In [10]:
# Stripping extra spaces in the description
data['Description'] = data['Description'].str.strip()
 
# Dropping the rows without any invoice number
data.dropna(axis = 0, subset =['InvoiceNo'], inplace = True)
data['InvoiceNo'] = data['InvoiceNo'].astype('str')
 
# Dropping all transactions which were done on credit
data = data[~data['InvoiceNo'].str.contains('C')]

In [11]:
# Transactions done in France
basket_France = (data[data['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [12]:
# Defining the hot encoding function to make the data suitable
# for the concerned libraries
def hot_encode(x):
    if(x<= 0):
        return 0
    if(x>= 1):
        return 1
 
# Encoding the datasets
basket_encoded = basket_France.applymap(hot_encode)
df = basket_encoded

In [13]:
df

Description,10 COLOUR SPACEBOY PEN,5 HOOK HANGER RED MAGIC TOADSTOOL,ALARM CLOCK BAKELIKE GREEN,ALARM CLOCK BAKELIKE ORANGE,ALARM CLOCK BAKELIKE PINK,ALARM CLOCK BAKELIKE RED,ASSORTED COLOUR MINI CASES,BASKET OF TOADSTOOLS,BIG DOUGHNUT FRIDGE MAGNETS,BIRD HOUSE HOT WATER BOTTLE,...,STRAWBERRY LUNCH BOX WITH CUTLERY,TABLECLOTH RED APPLES DESIGN,TEA BAG PLATE RED RETROSPOT,TEA PARTY BIRTHDAY CARD,TOOL BOX SOFT TOY,TRADITIONAL WOODEN CATCH CUP GAME,VINTAGE HEADS AND TAILS CARD GAME,VINTAGE SEASIDE JIGSAW PUZZLES,WOODLAND STICKERS,WOODLAND CHARLOTTE BAG
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
536974,0,0,0,0,0,0,1,0,1,0,...,0,0,0,1,0,0,0,0,0,0
537065,0,1,1,1,1,1,1,1,0,0,...,1,0,0,0,1,1,0,0,0,0
537463,0,0,0,0,0,0,0,0,1,0,...,1,1,1,1,0,0,0,0,1,1
537468,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [14]:
from mlxtend.frequent_patterns import apriori
import time

def apriori_sort(df, min_support, sort = "none"):
  start_time = time.time()
  dfa = apriori(df, min_support, use_colnames=True)
  if sort == "sup":
    print("Сортировка по support")
    dfa = dfa.sort_values(by=['support'])
  if sort == "items":
    print("Сортировка по itemsets")
    dfa = dfa.sort_values(by=['itemsets'])
  endtime  = time.time() - start_time
  return [dfa,endtime]


In [15]:
from mlxtend.frequent_patterns import association_rules

def apriori_association_sort(df, sort = "none", metric="confidence", min_threshold=0.7):
  start_time = time.time()
  dfaa = association_rules(df, metric, min_threshold)
  if sort == "sup":
    print("Сортировка по support")
    dfaa = dfaa.sort_values(by=['support'])
  if sort == "items":
    print("Сортировка по itemsets")
    dfaa = dfaa.sort_values(by=['itemsets'])
  endtime  = time.time() - start_time
  return [dfaa,endtime]

In [16]:
min_support = 0.17
sort = "none"
dfa_list = apriori_sort(df, min_support, sort)

print("Time of execution: ", dfa_list[1])
print("Min support: ", min_support)
dfa_list[0]

Time of execution:  0.08703422546386719
Min support:  0.17


,support,itemsets
0,0.333333,(ALARM CLOCK BAKELIKE GREEN)
1,0.333333,(ALARM CLOCK BAKELIKE PINK)
2,0.333333,(ALARM CLOCK BAKELIKE RED)
3,0.333333,(ASSORTED COLOUR MINI CASES)
4,0.333333,(BIG DOUGHNUT FRIDGE MAGNETS)
...,...,...
452,0.333333,"(SET OF SALT AND PEPPER TOADSTOOLS, SET/10 RED..."
453,0.333333,"(SET OF SALT AND PEPPER TOADSTOOLS, SET/10 RED..."
454,0.333333,"(SET OF SALT AND PEPPER TOADSTOOLS, SET/10 RED..."
455,0.333333,"(SET OF SALT AND PEPPER TOADSTOOLS, SET/10 RED..."


In [17]:
sort = "sup"
dfaa_list = apriori_association_sort(dfa_list[0],sort)

print("Time of execution: ", dfaa_list[1])
print("Min support: ", min_support)
dfaa_list[0]

Сортировка по support
Time of execution:  0.15144848823547363
Min support:  0.17


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
4756,"(SET OF SALT AND PEPPER TOADSTOOLS, ROUND SNAC...","(LUNCH BOX WITH CUTLERY RETROSPOT, POSTAGE)",0.333333,0.500000,0.333333,1.0,2.0,0.166667,inf
4755,"(SET OF SALT AND PEPPER TOADSTOOLS, LUNCH BOX ...","(POSTAGE, GUMBALL COAT RACK)",0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
4754,"(SET OF SALT AND PEPPER TOADSTOOLS, ROUND SNAC...","(LUNCH BOX WITH CUTLERY RETROSPOT, GUMBALL COA...",0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
4753,"(SET/10 RED POLKADOT PARTY CANDLES, ROUND SNAC...",(SET OF SALT AND PEPPER TOADSTOOLS),0.333333,0.500000,0.333333,1.0,2.0,0.166667,inf
...,...,...,...,...,...,...,...,...,...
427,(LUNCH BOX WITH CUTLERY RETROSPOT),"(SET OF SALT AND PEPPER TOADSTOOLS, POSTAGE)",0.500000,0.500000,0.500000,1.0,2.0,0.250000,inf
74,(ROUND SNACK BOXES SET OF4 WOODLAND),(POSTAGE),0.500000,1.000000,0.500000,1.0,1.0,0.000000,inf
75,(SET OF SALT AND PEPPER TOADSTOOLS),(POSTAGE),0.500000,1.000000,0.500000,1.0,1.0,0.000000,inf
56,(LUNCH BOX WITH CUTLERY RETROSPOT),(SET OF SALT AND PEPPER TOADSTOOLS),0.500000,0.500000,0.500000,1.0,2.0,0.250000,inf


In [18]:
min_support = 0.25
sort = "none"
dfa_list = apriori_sort(df, min_support, sort)

print("Time of execution: ", dfa_list[1])
print("Min support: ", min_support)
dfa_list[0]

Time of execution:  0.05988359451293945
Min support:  0.25


,support,itemsets
0,0.333333,(ALARM CLOCK BAKELIKE GREEN)
1,0.333333,(ALARM CLOCK BAKELIKE PINK)
2,0.333333,(ALARM CLOCK BAKELIKE RED)
3,0.333333,(ASSORTED COLOUR MINI CASES)
4,0.333333,(BIG DOUGHNUT FRIDGE MAGNETS)
...,...,...
452,0.333333,"(SET OF SALT AND PEPPER TOADSTOOLS, SET/10 RED..."
453,0.333333,"(SET OF SALT AND PEPPER TOADSTOOLS, SET/10 RED..."
454,0.333333,"(SET OF SALT AND PEPPER TOADSTOOLS, SET/10 RED..."
455,0.333333,"(SET OF SALT AND PEPPER TOADSTOOLS, SET/10 RED..."


In [19]:
sort = "sup"
dfaa_list = apriori_association_sort(dfa_list[0],sort)

print("Time of execution: ", dfaa_list[1])
print("Min support: ", min_support)
dfaa_list[0]

Сортировка по support
Time of execution:  0.04771065711975098
Min support:  0.25


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
4756,"(SET OF SALT AND PEPPER TOADSTOOLS, ROUND SNAC...","(LUNCH BOX WITH CUTLERY RETROSPOT, POSTAGE)",0.333333,0.500000,0.333333,1.0,2.0,0.166667,inf
4755,"(SET OF SALT AND PEPPER TOADSTOOLS, LUNCH BOX ...","(POSTAGE, GUMBALL COAT RACK)",0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
4754,"(SET OF SALT AND PEPPER TOADSTOOLS, ROUND SNAC...","(LUNCH BOX WITH CUTLERY RETROSPOT, GUMBALL COA...",0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
4753,"(SET/10 RED POLKADOT PARTY CANDLES, ROUND SNAC...",(SET OF SALT AND PEPPER TOADSTOOLS),0.333333,0.500000,0.333333,1.0,2.0,0.166667,inf
...,...,...,...,...,...,...,...,...,...
427,(LUNCH BOX WITH CUTLERY RETROSPOT),"(SET OF SALT AND PEPPER TOADSTOOLS, POSTAGE)",0.500000,0.500000,0.500000,1.0,2.0,0.250000,inf
74,(ROUND SNACK BOXES SET OF4 WOODLAND),(POSTAGE),0.500000,1.000000,0.500000,1.0,1.0,0.000000,inf
75,(SET OF SALT AND PEPPER TOADSTOOLS),(POSTAGE),0.500000,1.000000,0.500000,1.0,1.0,0.000000,inf
56,(LUNCH BOX WITH CUTLERY RETROSPOT),(SET OF SALT AND PEPPER TOADSTOOLS),0.500000,0.500000,0.500000,1.0,2.0,0.250000,inf


In [20]:
min_support = 0.35
sort = "none"
dfa_list = apriori_sort(df, min_support, sort)

print("Time of execution: ", dfa_list[1])
print("Min support: ", min_support)
dfa_list[0]

Time of execution:  0.013689279556274414
Min support:  0.35


,support,itemsets
0,0.5,(LUNCH BAG RED RETROSPOT)
1,0.5,(LUNCH BOX WITH CUTLERY RETROSPOT)
2,1.0,(POSTAGE)
3,0.5,(ROUND SNACK BOXES SET OF4 WOODLAND)
4,0.5,(SET OF SALT AND PEPPER TOADSTOOLS)
5,0.5,"(POSTAGE, LUNCH BAG RED RETROSPOT)"
6,0.5,"(LUNCH BOX WITH CUTLERY RETROSPOT, POSTAGE)"
7,0.5,"(SET OF SALT AND PEPPER TOADSTOOLS, LUNCH BOX ..."
8,0.5,"(ROUND SNACK BOXES SET OF4 WOODLAND, POSTAGE)"
9,0.5,"(SET OF SALT AND PEPPER TOADSTOOLS, POSTAGE)"


In [21]:
sort = "sup"
dfaa_list = apriori_association_sort(dfa_list[0],sort)

print("Time of execution: ", dfaa_list[1])
print("Min support: ", min_support)
dfaa_list[0]

Сортировка по support
Time of execution:  0.013611555099487305
Min support:  0.35


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(LUNCH BAG RED RETROSPOT),(POSTAGE),0.5,1.0,0.5,1.0,1.0,0.00,inf
1,(LUNCH BOX WITH CUTLERY RETROSPOT),(POSTAGE),0.5,1.0,0.5,1.0,1.0,0.00,inf
2,(SET OF SALT AND PEPPER TOADSTOOLS),(LUNCH BOX WITH CUTLERY RETROSPOT),0.5,0.5,0.5,1.0,2.0,0.25,inf
3,(LUNCH BOX WITH CUTLERY RETROSPOT),(SET OF SALT AND PEPPER TOADSTOOLS),0.5,0.5,0.5,1.0,2.0,0.25,inf
4,(ROUND SNACK BOXES SET OF4 WOODLAND),(POSTAGE),0.5,1.0,0.5,1.0,1.0,0.00,inf
5,(SET OF SALT AND PEPPER TOADSTOOLS),(POSTAGE),0.5,1.0,0.5,1.0,1.0,0.00,inf
6,"(SET OF SALT AND PEPPER TOADSTOOLS, LUNCH BOX ...",(POSTAGE),0.5,1.0,0.5,1.0,1.0,0.00,inf
7,"(SET OF SALT AND PEPPER TOADSTOOLS, POSTAGE)",(LUNCH BOX WITH CUTLERY RETROSPOT),0.5,0.5,0.5,1.0,2.0,0.25,inf
8,"(LUNCH BOX WITH CUTLERY RETROSPOT, POSTAGE)",(SET OF SALT AND PEPPER TOADSTOOLS),0.5,0.5,0.5,1.0,2.0,0.25,inf
9,(SET OF SALT AND PEPPER TOADSTOOLS),"(LUNCH BOX WITH CUTLERY RETROSPOT, POSTAGE)",0.5,0.5,0.5,1.0,2.0,0.25,inf


In [22]:
min_support = 0.45
sort = "none"
dfa_list = apriori_sort(df, min_support, sort)

print("Time of execution: ", dfa_list[1])
print("Min support: ", min_support)
dfa_list[0]

Time of execution:  0.027666807174682617
Min support:  0.45


,support,itemsets
0,0.5,(LUNCH BAG RED RETROSPOT)
1,0.5,(LUNCH BOX WITH CUTLERY RETROSPOT)
2,1.0,(POSTAGE)
3,0.5,(ROUND SNACK BOXES SET OF4 WOODLAND)
4,0.5,(SET OF SALT AND PEPPER TOADSTOOLS)
5,0.5,"(POSTAGE, LUNCH BAG RED RETROSPOT)"
6,0.5,"(LUNCH BOX WITH CUTLERY RETROSPOT, POSTAGE)"
7,0.5,"(SET OF SALT AND PEPPER TOADSTOOLS, LUNCH BOX ..."
8,0.5,"(ROUND SNACK BOXES SET OF4 WOODLAND, POSTAGE)"
9,0.5,"(SET OF SALT AND PEPPER TOADSTOOLS, POSTAGE)"


In [23]:
sort = "sup"
dfaa_list = apriori_association_sort(dfa_list[0],sort)

print("Time of execution: ", dfaa_list[1])
print("Min support: ", min_support)
dfaa_list[0]

Сортировка по support
Time of execution:  0.012130260467529297
Min support:  0.45


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(LUNCH BAG RED RETROSPOT),(POSTAGE),0.5,1.0,0.5,1.0,1.0,0.00,inf
1,(LUNCH BOX WITH CUTLERY RETROSPOT),(POSTAGE),0.5,1.0,0.5,1.0,1.0,0.00,inf
2,(SET OF SALT AND PEPPER TOADSTOOLS),(LUNCH BOX WITH CUTLERY RETROSPOT),0.5,0.5,0.5,1.0,2.0,0.25,inf
3,(LUNCH BOX WITH CUTLERY RETROSPOT),(SET OF SALT AND PEPPER TOADSTOOLS),0.5,0.5,0.5,1.0,2.0,0.25,inf
4,(ROUND SNACK BOXES SET OF4 WOODLAND),(POSTAGE),0.5,1.0,0.5,1.0,1.0,0.00,inf
5,(SET OF SALT AND PEPPER TOADSTOOLS),(POSTAGE),0.5,1.0,0.5,1.0,1.0,0.00,inf
6,"(SET OF SALT AND PEPPER TOADSTOOLS, LUNCH BOX ...",(POSTAGE),0.5,1.0,0.5,1.0,1.0,0.00,inf
7,"(SET OF SALT AND PEPPER TOADSTOOLS, POSTAGE)",(LUNCH BOX WITH CUTLERY RETROSPOT),0.5,0.5,0.5,1.0,2.0,0.25,inf
8,"(LUNCH BOX WITH CUTLERY RETROSPOT, POSTAGE)",(SET OF SALT AND PEPPER TOADSTOOLS),0.5,0.5,0.5,1.0,2.0,0.25,inf
9,(SET OF SALT AND PEPPER TOADSTOOLS),"(LUNCH BOX WITH CUTLERY RETROSPOT, POSTAGE)",0.5,0.5,0.5,1.0,2.0,0.25,inf


In [24]:
min_support = 0.6
sort = "none"
dfa_list = apriori_sort(df, min_support, sort)

print("Time of execution: ", dfa_list[1])
print("Min support: ", min_support)
dfa_list[0]

Time of execution:  0.011763572692871094
Min support:  0.6


,support,itemsets
0,1.0,(POSTAGE)


In [25]:
sort = "sup"
dfaa_list = apriori_association_sort(dfa_list[0],sort)

print("Time of execution: ", dfaa_list[1])
print("Min support: ", min_support)
dfaa_list[0]

Сортировка по support
Time of execution:  0.010179758071899414
Min support:  0.6


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
